### How does the popularity of authors within a playlist affect the popularity of the playlist itself?

In [2]:
import os
import json
import pickle
from collections import defaultdict
from collections import Counter
import numpy as np

In [3]:
MPD = 'mpd/'
playlist_files = [os.path.join(MPD, x) for x in os.listdir(MPD)]

In [4]:
SONG_DATA = 'mappings/songs'
song_files = [os.path.join(SONG_DATA, x) for x in os.listdir(SONG_DATA)]

In [5]:
ARTIST_DATA = 'mappings/artists/artist_matching.pkl'
with open(ARTIST_DATA, 'rb') as fp:
    artist_dict = pickle.load(fp)

In [7]:
with open('mappings/Q2/auth_to_derived_popularity.pkl', 'rb') as fp:
    artistID_to_popularity = pickle.load(fp)

In [8]:
songID_to_popularity = {}
for file in song_files:
    with open(file, 'rb') as fp:
        data = pickle.load(fp)
    for songID,song in data.items():
        total_popularity = 0
        num_authors = len(song['artists'])
        for artist in song['artists']:
            artistID = artist['id']
            popularity = artistID_to_popularity[artistID]
            total_popularity += popularity
        songID_to_popularity[songID] = (total_popularity, num_authors)

In [9]:
list(songID_to_popularity.items())[:5]

[('spotify:track:1fyTBapjw8q9MlzLVdU6wg', (5.107682933238617, 1)),
 ('spotify:track:4cC9nQMgClmz8cix6l5CSX', (0.01990241312987063, 1)),
 ('spotify:track:3qwh1awyjkXFUER1Pp64qv', (0.3112013689397953, 1)),
 ('spotify:track:6NHONOfLaDTtSG8WtCTpex', (0.0407094814020081, 1)),
 ('spotify:track:7HxecasMeh6aCAvQPGiFgP', (0.017188447703070087, 1))]

In [10]:
playlist_data = {} # list, num_followers
for file in playlist_files:
    with open(file, 'rb') as fp:
        data = json.load(fp)
    for playlist in data['playlists']:
        empty = False
        ID = playlist['pid']
        author_data = []
        for track in playlist['tracks']:
            tID = track['track_uri']
            if tID not in songID_to_popularity:
                continue
            author_data.append(songID_to_popularity[tID])
        followers = playlist['num_followers']
        playlist_data[ID] = (author_data, followers)

In [11]:
list(playlist_data.items())[:5]

[(962000,
  ([(0.06423051510094611, 1),
    (0.16917051160390034, 1),
    (0.003618620569067387, 1),
    (0.012665171991735854, 1),
    (0.15198206390083024, 1),
    (0.02985361969480594, 1),
    (0.0018093102845336934, 1),
    (0.01628379256080324, 1),
    (0.4568508468447576, 1),
    (0.14836344333176285, 1),
    (0.01899775798760378, 1),
    (0.16917051160390034, 1),
    (0.021711723414404322, 1),
    (0.10584465164522107, 1),
    (0.2496848192656497, 1),
    (0.20807068272137474, 1),
    (0.012665171991735854, 1),
    (0.14384016762042862, 1),
    (0.061516549674145576, 1),
    (0.05880258424734504, 1),
    (0.10584465164522107, 1),
    (0.16917051160390034, 1),
    (0.03166292997933964, 1),
    (0.012665171991735854, 1),
    (0.033472240263873326, 1),
    (0.06694448052774665, 1),
    (0.0289489645525391, 2),
    (0.061516549674145576, 1),
    (0.009951206564935314, 1),
    (0.04975603282467657, 1),
    (0.82052221403603, 1),
    (0.061516549674145576, 1),
    (0.5744560153394477,

In [12]:
# with open('mappings/Q2/playlist_data.pkl', 'wb') as fp:
#     pickle.dump(playlist_data, fp)

In [13]:
for k, v in playlist_data.items():
    data = v[0]
    for x in data:
        if len(x) != 2:
            print(k)

In [14]:
with open('mappings/Q2/playlist_dataDERIVEDPOP.pkl', 'wb') as fp:
    pickle.dump(playlist_data, fp)